<!--NAVIGATION-->
< [Output widgets: leveraging Jupyter's display system](04.01-more-on-output-widget.ipynb) | [Contents](00.00-index.ipynb) | [*OPTIONAL* Three approaches to events](05.01-OPTIONAL-Widget_Events_2.ipynb) >

# Widgets Events

In [ ]:
import ipywidgets as widgets
from IPython.display import display

## Traitlets events

Every widget class is an `HasTraits` class, which means they benefit from the Traitlets API concerning the validation and observation of properties (see https://traitlets.readthedocs.io/en/stable/using_traitlets.html#using-traitlets).

### Trait class example: properties **validation** and **observation**

Traitlets are validated by **type** and **value**:

In [ ]:
from traitlets import HasTraits, Unicode, Int, TraitError, validate

class Identity(HasTraits):

    username = Unicode()
    age = Int()

    @validate('age')
    def _validate_age(self, proposal):
        age = proposal['value']

        if age < 0:
            raise TraitError('age can not be negative')

        if age > 115:
            raise TraitError('this is too old to be true')

        return age

In [ ]:
jane = Identity(username=999, age=25)
jane.age

In [ ]:
jane.age = 32

Every `HasTraits` class has an `observe` method which allows observing properties changes. You can assign a Python callback function that will be called when a property changes.

The callback handler passed to observe will be called with one change argument. The change object holds at least a `type` key and a `name` key, corresponding respectively to the type of notification and the name of the attribute that triggered the notification.

Other keys may be passed depending on the value of `type`. In the case where type is `change`, we also have the following keys:

- `owner` : the HasTraits instance
- `old` : the old value of the modified trait attribute
- `new` : the new value of the modified trait attribute
- `name` : the name of the modified trait attribute.

In [ ]:
HasTraits.observe?

In [ ]:
# We use an output widget here for capturing the print calls and showing them at the right place in the Notebook
output = widgets.Output()

@output.capture()
def print_change(change):
    print(change)

# Observe jane.age changes, and print them
jane.observe(print_change, 'age')

output

In [ ]:
jane.age = 39

### Registering callbacks to trait changes in the kernel

Since `Widget` classes inherit from `HasTraits`, you can register handlers to the change events whenever the model gets updates from the front-end.

In [ ]:
widgets.Widget.observe?

In [ ]:
caption = widgets.Label(value='Start moving the slider!')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    sign = 'negative' if change.new < 0 else 'positive'
    caption.value = f'The slider value is {sign}'

slider.observe(handle_slider_change, names='value')

display(caption, slider)

### Callback signatures

Mentioned in the doc string, the callback registered must have the signature `handler(change)` where `change` is a dictionary holding the information about the change. 

Using this method, an example of how to output an `IntSlider`'s value as it is changed can be seen below.

In [ ]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)

def on_value_change(change):
    output2.clear_output(wait=True)

    old = change['old']
    new = change['new']

    with output2:
        print(f'The value was {old} and is now {new}')

int_range.observe(on_value_change, names='value')